In [18]:
import requests
import urllib3
import json
from lxml import html, etree
import requests.packages.urllib3.exceptions
from urllib3.exceptions import InsecureRequestWarning
import time
import re
import datetime

In [4]:
def getRequest(url):
    headers = {
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'en-GB,en;q=0.9,en-US;q=0.8,ml;q=0.7',
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.140 Safari/537.36'
    }
    requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
    response = requests.get(url, verify=False, headers=headers)
    return response.text

In [5]:
urlsList = [('China', 'https://www.theguardian.com/world/china')]

In [7]:
urls = []
for urlPair in urlsList:
  #https://www.theguardian.com/world/china?page=1100
  for i in range(0, 1100):
    url = urlPair[1]
    pagesParser = html.fromstring(getRequest(url))

    page = i
    #https://www.theguardian.com/world/china?page=1000
    parser = html.fromstring(getRequest(url + "?page=" + str(page)))

    #<div class="fc-item__content "> 
    items = parser.xpath("//div[@class='fc-item__content ']")

    for item in items:
      for elt in item.xpath(".//h3[@class='fc-item__title']"):
        urls.append(elt[0].attrib['href'])

print(len(urls))

19730


In [54]:
def month_converter(month):
    months = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
    return months.index(month) + 1

def dateToList(dateList):
    dateList[0] = int(dateList[0])
    dateList[1] = month_converter(dateList[1])
    dateList[2] = int(dateList[2])
    return dateList

def isYear(string):
    if re.search("(?=.*[1][9][0-9][0-9]|[2][0-9][0-9][0-9])", string) is not None:
        return True
    else:
        return False

In [ ]:
dfRows = []
for url in urls:
    elements = url.rstrip().split('/')
    for i in range(0, len(elements)):
        if elements[i].isnumeric() and isYear(elements[i]):
            before = elements[2:i]
            #ignoring non-articles with little or no text
            if "video" in before:
                continue
            elif "live" in before:
                print("------------------------------\n" + url)
                continue
            elif "gallery" in before:
                continue
            elif "audio" in before:
                continue
            elif "picture" in before:
                continue
            elif "blog" in before:
                print("------------------------------\n" + url)
                continue
            #elif "commentisfree"in before:
            #    print("------------------------------\n" + url)
                continue 
            elif "shortcuts" in before:
                print("------------------------------\n" + url)
                continue
            elif "from-the-archive-blog" in before:
                print("------------------------------\n" + url)
                continue
            elif "radical-conservation" in before:
                print("------------------------------\n" + url)
                continue
            elif "lostinshowbiz" in before:
                print("------------------------------\n" + url)
                continue
            elif "the-running-blog" in before:
                print("------------------------------\n" + url)
                continue
            elif "australia-books-blog" in before:
                print("------------------------------\n" + url)
                continue
        
            data = {}
            newsParser = html.fromstring(getRequest(url))
            #print(url)
            #print(newsParser.xpath(".//h1")[0].text.strip())

            #interactive articles that we don't need
            if "ng-interactive" in url:
                continue
            #specially formatted Guardian article
            elif newsParser.xpath(".//h1")[0].text == None:
                for elt in newsParser.xpath(".//h1"):
                    #print(url)
                    data["title"] = elt[0].text.strip()
            #normal Guardian article
            else:
                data["title"] = newsParser.xpath(".//h1")[0].text.strip()

            #when we get kicked out the h1 (title) changes
            if '429 Too Many Requests' in data["title"]:
                print("Too many requests!")
                print("Data gathered: ", len(dfRows))
                break

            data["url"] = url
            #date always starts from i
            data["date"] = dateToList(elements[i:i+3])
            
            #collect all paragraphs
            paragraphList = []
            for item in newsParser:
                for elt in item.xpath(".//p"):
                    paragraphList.append(elt.text)

            #first one is always the subtitle
            data["subtitle"] = paragraphList.pop(0)

            #collect raw article text
            data["text"] = ''
            for paragraph in paragraphList:
                #print(paragraph)
                if isinstance(paragraph, str) is False:
                    continue
                elif "Last modified on " in paragraph:
                    continue
                try:
                    data["text"] = data["text"] + paragraph.strip()
                except TypeError:
                    print("Type Error!")
                    print(paragraph)
            
            #write to list and continue after a delay
            #print(data)
            dfRows.append(data)
            time.sleep(800/1000)
        
print("Scraping finished!")

------------------------------
https://www.theguardian.com/australia-news/live/2021/mar/31/australia-news-live-queensland-hospital-lockdown-covid-outbreak-nsw-victoria-coronavirus
------------------------------
https://www.theguardian.com/world/live/2021/mar/26/china-sanctions-uighurs-uk-covid-coronavirus-boris-johnson-latest-updates
------------------------------
https://www.theguardian.com/politics/live/2021/mar/22/uk-covid-live-news-boris-johnson-vaccine-exports-coronavirus-latest-updates
------------------------------
https://www.theguardian.com/politics/live/2021/mar/17/uk-politics-live-dominic-raab-covid-coronavirus-dominic-cummings-latest-updates
------------------------------
https://www.theguardian.com/politics/live/2021/mar/16/uk-covid-live-new-novel-pandemic-realistic-possibility-boris-johnson-defence-review-latest-upates
------------------------------
https://www.theguardian.com/business/live/2021/mar/15/china-industrial-output-retail-sales-covid-19-markets-ftse-recovery-ba

In [46]:
#date = "Last modified on Wed 7 Apr 2021 16.37 BST"
#print(dateToList(date))

#newsParser = html.fromstring(getRequest("https://www.theguardian.com/world/ng-interactive/2020/dec/14/coronavirus-2020-timeline-covid-19"))
#newsParser = html.fromstring(getRequest("https://www.theguardian.com/world/2020/oct/26/china-new-coronavirus-outbreak-detected-in-xinjiang-city-of-kashgar"))
#print(newsParser.xpath(".//h1"))
#for item in newsParser:
#      for elt in item.xpath(".//p"):
#        print(elt.text)

19730


In [14]:
import pandas as pd
import numpy as np

# create a new dataframe and parser
df = pd.DataFrame()
df = df.append(dfRows, ignore_index=True)
print(len(dfRows))
df.tail(50)

8571


,title,url,date,subtitle,text
8521,Chinese put Britain in its place ahead of Li K...,https://www.theguardian.com/world/2014/jun/15/...,"[2014, 6, 15]",With a date for tea with the Queen in the bag ...,Britain now ranks behindThe remarks by the amb...
8522,Chinese premier's visit to UK holds promise of...,https://www.theguardian.com/world/2014/jun/15/...,"[2014, 6, 15]",A three-day visit to Britain by the Chinese pr...,In his first visit to Britain since becoming p...
8523,Pu Zhiqiang: China arrests human rights defender,https://www.theguardian.com/world/2014/jun/14/...,"[2014, 6, 14]",Government takes further steps to silence cele...,"Police inPu Zhiqiang, a prominent campaigner w..."
8524,Britain to relax visa rules for Chinese visitors,https://www.theguardian.com/politics/2014/jun/...,"[2014, 6, 13]",Britain is to announce that it will liberalise...,As the Chinese ambassador to London warned tha...
8525,Ai Weiwei and David Hockney pool data for art ...,https://www.theguardian.com/technology/2014/ju...,"[2014, 6, 13]",Artists and technologists are collaborating in...,"Ai Weiwei's voice is shrill and rhythmic, echo..."
8526,"Fewer elephants killed in 2013, figures show",https://www.theguardian.com/environment/2014/j...,"[2014, 6, 13]",The number of African elephants slaughtered fo...,"The""Although the sharp upward trend in illegal..."
8527,Is that leg loaded? Ai Weiwei starts web craze...,https://www.theguardian.com/artanddesign/2014/...,"[2014, 6, 13]",None,"The question, of course, is what Ai Weiwei int..."
8528,Inside Shenzhen: China's Silicon Valley,https://www.theguardian.com/cities/2014/jun/13...,"[2014, 6, 13]",Just 30 years ago this Pearl River Delta megac...,"In pictures:Everywhere you look, there is ente..."
8529,"Momo, the Chinese app that exposes sex and gen...",https://www.theguardian.com/technology/2014/ju...,"[2014, 6, 12]",When Chen Xiaozhe downloaded the smartphone ap...,"To attract potential dates, Chen updates his p..."
8530,The Chinese billionaire with six months to mak...,https://www.theguardian.com/artanddesign/archi...,"[2014, 6, 12]",Zhang Yue was the first man in China to own a ...,“It's always said that the solution to environ...


In [23]:
#print(len(df['tiltle']==None))
#print(len(df['url']==None))
#print(len(df['subtitle']==None))
#print(len(df['text']==None))
#print(df[df['url']=='https://www.theguardian.com/football/2019/dec/13/arsenal-distance-themselves-from-mesut-ozil-comments-china-uighur-people'])
#print(df[df['title']=='429 Too Many Requests'])
#temp = "https://www.theguardian.com/books/australia-books-blog/2019/apr/29/les-murray-remembered-by-john-kinsella-a-brilliant-battler-poet".split('/')
#temp = "https://www.theguardian.com/artanddesign/gallery/2015/feb/13/red-army-primary-school-gallery".split('/')#[5:8]
#print(temp)

['https:', '', 'www.theguardian.com', 'books', 'australia-books-blog', '2019', 'apr', '29', 'les-murray-remembered-by-john-kinsella-a-brilliant-battler-poet']
